# SVD Implementation
write python code to compute SVD

In [1]:
import numpy as np
from sympy import symbols
from sympy.solvers import solve

In [2]:
# make orthogonal matrix by using exisiting orthogonal vectors in matrix
def make_orthogonal(matrix, non_zero_eig):
    dim = matrix.shape[0]
    
    # The matrix is already orthogonal
    if non_zero_eig == dim:
        return matrix
    
    # Create dim amount of variables
    v = symbols('v0:'+str(dim))
    
    # Every eigenvector that correspond to a non-zero eigenvalue forms a linear equation
    eq = [0]*non_zero_eig
    
    # The new orthogonal vector must have a dot product of zero with other exist vector in matrix
    for i in range(0,non_zero_eig):
        for j in range(0,dim):
            eq[i] += matrix[:,i][j]*v[j] 
     
    # Solve the system of linear equations
    eq_tup = tuple(eq)
    ans = solve(tuple(eq),v) 
    unknown_dim = dim - non_zero_eig
    
    # Variables that need to be used for substitution
    sub = []
    for i in range(1, unknown_dim+1):
        sub.append((v[-1*i],1))
    
    # Free variables will be set to 1 automatically
    vec = np.ones((dim,1))
    for i in range(0, non_zero_eig):
        vec[i] = ans[v[i]].subs(sub)
    matrix[:,non_zero_eig:non_zero_eig+1] = vec/np.linalg.norm(vec)
    
    # Every recursion of make_orthogonal add one more orthogonal basis to the matrix
    return make_orthogonal(matrix, non_zero_eig+1)

In [3]:
def my_SVD(A):
    # rows of A
    m = A.shape[0]
    # columns of A
    n = A.shape[1]
    
    # Find Eigenvectors of AA_T
    Uw, U = np.linalg.eig(A.dot(A.T))
    # Sort the eigenvalue and eigenvector
    idx = Uw.argsort()[::-1]   
    Uw = Uw[idx]
    U = U[:,idx]
    
    # Find Eigenvectors of A_TA
    Vw, V = np.linalg.eig(A.T.dot(A))
    # Sort the eigenvalue and eigenvector
    idx = Vw.argsort()[::-1]   
    Vw = Vw[idx]
    V = V[:,idx]

    # Choose the smaller dimension size
    w = Uw if m<n else Vw
    # Make small negative value zero
    w[np.where(w<0)] = 1e-30
    w = np.sqrt(w)
    # Find the amount of non-zero singular value
    non_zero_eig = w[np.where(w>1e-10)].size
    
    # Sigma
    S = np.zeros((m,n))
    np.fill_diagonal(S, w)
    
    # w * w_r = 1
    w_r = np.reciprocal(w)
    if m>n:
        # Inverse of Sigma
        S_inv = np.zeros((n,m))
        np.fill_diagonal(S_inv, w_r)
        # Calculalte U by A V S_inv
        U = A.dot(V).dot(S_inv)
        
    else:
        # Inverse of Sigma_T
        S_T_inv = np.zeros((m,n))
        np.fill_diagonal(S_T_inv, w_r)
        # Calculate V by A_T U S_T_inv
        V = A.T.dot(U).dot(S_T_inv)
        
    # Make U orthogonal
    U = make_orthogonal(U, non_zero_eig)
    
    # Make V orthogonal
    V = make_orthogonal(V, non_zero_eig)
    
    return U, S, V.T

### Examine the correctness of my_SVD

In [4]:
a = np.array([[1,2],
              [3,4],
              [5,6],
              [9,10]])
aU, aS, aV_T = my_SVD(a)
print("U:")
print(aU)
print("S:")
print(aS)
print("V_T:")
print(aV_T)
print("Examine SVD")
print("USV_T (Should be a):")
print(aU.dot(aS).dot(aV_T))
print("Examine the orthogonal property")
print("UU_T (Should be I):")
print(aU.dot(aU.T))
print("U_TU (Should be I)")
print(aU.T.dot(aU))
print("VV_T (Should be I):")
print(aV_T.T.dot(aV_T))
print("V_TV (Should be I)")
print(aV_T.dot(aV_T.T))

U:
[[-0.1315692   0.76334104  0.54433105  0.32203059]
 [-0.30275977  0.47182861 -0.81649658  0.13801311]
 [-0.47395035  0.18031617  0.13608276 -0.85108086]
 [-0.81633151 -0.4027087   0.13608276  0.39103715]]
S:
[[16.47678113  0.        ]
 [ 0.          0.71811111]
 [ 0.          0.        ]
 [ 0.          0.        ]]
V_T:
[[-0.65283284 -0.757502  ]
 [-0.757502    0.65283284]]
Examine SVD
USV_T (Should be a):
[[ 1.  2.]
 [ 3.  4.]
 [ 5.  6.]
 [ 9. 10.]]
Examine the orthogonal property
UU_T (Should be I):
[[ 1.00000000e+00 -1.15116250e-14 -4.88498131e-15  8.99280650e-15]
 [-1.15116250e-14  1.00000000e+00 -3.35842465e-15  5.55805402e-15]
 [-4.88498131e-15 -3.35842465e-15  1.00000000e+00  2.27595720e-15]
 [ 8.99280650e-15  5.55805402e-15  2.27595720e-15  1.00000000e+00]]
U_TU (Should be I)
[[ 1.00000000e+00  1.38777878e-15 -1.66533454e-16 -3.33066907e-16]
 [ 1.38777878e-15  1.00000000e+00 -1.31838984e-16  2.22044605e-16]
 [-1.66533454e-16 -1.31838984e-16  1.00000000e+00 -1.52655666e-16]
 

In [5]:
b = np.array([[1,2,3,4],
              [3,4,5,6],
              [7,8,9,10]])
bU, bS, bV_T = my_SVD(b)
print("U:")
print(bU)
print("S:")
print(bS)
print("V_T:")
print(bV_T)
print("Examine SVD")
print("USV_T (Should be a):")
print(bU.dot(bS).dot(bV_T))
print("Examine the orthogonal property")
print("UU_T (Should be I):")
print(bU.dot(bU.T))
print("U_TU (Should be I)")
print(bU.T.dot(bU))
print("VV_T (Should be I):")
print(bV_T.T.dot(bV_T))
print("V_TV (Should be I)")
print(bV_T.dot(bV_T.T))

U:
[[-0.26326666 -0.80310422  0.53452248]
 [-0.45845408 -0.38335717 -0.80178373]
 [-0.84882891  0.45613693  0.26726124]]
S:
[[2.01804322e+01 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.65835898e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e-15 0.00000000e+00]]
V_T:
[[-0.37563275 -0.45345812 -0.5312835  -0.60910887]
 [ 0.74759617  0.30720633 -0.13318351 -0.57357335]
 [ 0.5        -0.83333333  0.16666667  0.16666667]
 [ 0.2236068   0.0745356  -0.81989159  0.52174919]]
Examine SVD
USV_T (Should be a):
[[ 1.  2.  3.  4.]
 [ 3.  4.  5.  6.]
 [ 7.  8.  9. 10.]]
Examine the orthogonal property
UU_T (Should be I):
[[ 1.00000000e+00 -1.11022302e-16  0.00000000e+00]
 [-1.11022302e-16  1.00000000e+00  1.94289029e-16]
 [ 0.00000000e+00  1.94289029e-16  1.00000000e+00]]
U_TU (Should be I)
[[ 1.00000000e+00  2.22044605e-16  5.55111512e-17]
 [ 2.22044605e-16  1.00000000e+00 -4.99600361e-16]
 [ 5.55111512e-17 -4.99600361e-16  1.00000000e+00]]
VV_T (